# Build End-to-End LLM project for a retail domain (t-shirts selling store).

Tech stack for this project
* UI: Streamlit
* LLM: Google Gemini LLM model
* Embeddings: Hugging Face
* Framework: Langchain

# Importing required modules

In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.utilities import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool
from langchain import hub
from dotenv import load_dotenv
import os
import getpass

# Insert API key

In [2]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

# Initializing Gemini AI model

In [3]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

# Establishing database connection

In [4]:
db_user = "root"
db_password = "Kitsune#sql98"
db_host = "localhost"
db_name = "bank_data"

# Name of test databases
# "atliq_tshirts"
# "bank_data"

In [5]:
connection_Uri = f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}"
db = SQLDatabase.from_uri(connection_Uri)

In [6]:
print(db.table_info)
print("--------")
print("--------")
print(f"Dialect: {db.dialect}")
print(f"Available tables: {db.get_usable_table_names()}")


CREATE TABLE bankers (
	banker_id INTEGER, 
	banker_name VARCHAR(7) CHARACTER SET utf8mb3 COLLATE utf8mb3_general_ci, 
	branch_email VARCHAR(19) CHARACTER SET utf8mb3 COLLATE utf8mb3_general_ci, 
	branch_id INTEGER
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from bankers table:
banker_id	banker_name	branch_email	branch_id
1	Rosana	rosana@abcbank.com	2
2	Carina	carina@abcbank.com	5
3	Gloria	gloria@abcbank.com	3
*/


CREATE TABLE branch (
	branch_id INTEGER, 
	branch_name VARCHAR(15) CHARACTER SET utf8mb3 COLLATE utf8mb3_general_ci, 
	branch_city VARCHAR(9) CHARACTER SET utf8mb3 COLLATE utf8mb3_general_ci
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from branch table:
branch_id	branch_name	branch_city
1	Clear Water Bay	Hong Kong
2	Downtown	Edmonton
3	Howe	Vancouver
*/


CREATE TABLE credit_card (
	credit_card_number VARCHAR(19) CHARACTER SET utf8mb3 COLLATE utf8mb3_general_ci, 
	customer_id INTEGER, 
	currency VARCHAR(3) C

# Core functionality

## SQL Query Generation

In [7]:
"""
1. Pulls a specialized SQL generation prompt from LangChain Hub
2. Formats the prompt with database schema information
3. Sends the prompt to Gemini AI for query generation
4. Uses a second prompt to extract just the SQL from the response
"""

query_prompt_template = hub.pull("langchain-ai/sql-query-system-prompt")

def write_query(question: str):
    """Generate SQL query from the user's question."""
    prompt = query_prompt_template.invoke(
        {
            "dialect": db.dialect,
            "top_k": 10,
            "table_info": db.get_table_info(),
            "input": question,
        }
    )
    response = llm.invoke(prompt.to_string())
    extraction_prompt = """
    Please extract the SQL query from the following text and return only the SQL query without any additional characters or formatting:

    {response}

    SQL Query:
    """
    # Format the prompt with the actual response
    prompt = extraction_prompt.format(response=response)
    # Invoke the language model with the prompt
    parsed_query = llm.invoke(prompt)
    return parsed_query.content

d:\projects\mysql_llm_agent\venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


## Query Execution

In [8]:
"""
This function creates a query execution tool and runs the generated SQL against your database, returning the raw results.
"""

def execute_query(query: str):
    """Execute the SQL query."""
    execute_query_tool = QuerySQLDatabaseTool(db=db)
    return execute_query_tool.invoke(query)

## Natural Language Answer Generation

In [9]:
"""
This function takes the original question, generated SQL, and query results, then asks Gemini to formulate a human-friendly answer.
"""

def generate_answer(question: str, query: str, result: str):
    """Generate an answer using the query results."""
    prompt = (
        "Given the following user question, corresponding SQL query, "
        "and SQL result, answer the user question.\n\n"
        f'Question: {question}\n'
        f'SQL Query: {query}\n'
        f'SQL Result: {result}'
    )
    response = llm.invoke(prompt)
    return response.content

# Putting It All Together

In [10]:
def model_output(input):
    query = write_query(input)
    result = execute_query(query) # type: ignore
    answer = generate_answer(input, query, result) # type: ignore

    print(f"Query: \n\n{query}")
    print("\n")
    print(answer)

In [11]:
# Questions for the "altiq_tshirts" database
"""
"How many t-shirts do we have left for nike in extra small size and white color?"
"How much is the price of the inventory for all small size t-shirts?"
"If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?"
"If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?"
"""

# question = string_value

'\n"How many t-shirts do we have left for nike in extra small size and white color?"\n"How much is the price of the inventory for all small size t-shirts?"\n"If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?"\n"If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?"\n'

In [20]:
# Questions for the "bank_data" database
"""
"What is the total balance of all the accounts in the database?"
"Which account has the lowest balance and what is the client name?"
"Name of the client who has the highest credit card limit and what is the limit"
"Name of the client with most number of credit cards, how many and total limit."
"Name of the client with the second top number of credit cards, how many and total limit."
"""

question = "Para clientes con ahorros mayores a cero, proporciona: Nombre, código de cliente, cantidad de tarjetas de crédito, límite total de crédito." \
"Ordena los datos de límite total en orden descendente y muestra los resultados en formato tabular."

In [21]:
model_output(question)

Query: 

SELECT
  T1.customer_name,
  T1.customer_id,
  COUNT(T2.credit_card_number) AS num_credit_cards,
  SUM(T2.credit_limit) AS total_credit_limit
FROM customer AS T1
INNER JOIN credit_card AS T2
  ON T1.customer_id = T2.customer_id
INNER JOIN savings_account AS T3
  ON T1.customer_id = T3.customer_id
WHERE
  T3.balance_savings > 0
GROUP BY
  T1.customer_id,
  T1.customer_name
ORDER BY
  total_credit_limit DESC
LIMIT 10;


```
| customer_name | customer_id | num_credit_cards | total_credit_limit |
|---|---|---|---|
| David | 4 | 1 | 20000 |
| Paul | 8 | 1 | 20000 |
| Amy | 1 | 2 | 15000 |
| Ricky | 9 | 2 | 13000 |
| John | 5 | 1 | 10000 |
| Linda | 6 | 1 | 10000 |
| Mandy | 7 | 1 | 10000 |
| Bob | 2 | 1 | 5000 |
| Carson | 3 | 1 | 3000 |
```
